<a href="https://colab.research.google.com/github/Janamawasy/ML/blob/main/draft_titanic_comp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from datetime import datetime
tf.__version__

'2.15.0'

In [59]:
# -------------- code - acc=0.85   score-0.76

train = pd.read_csv('train (1).csv')
test = pd.read_csv('test (1).csv')
y_test_data = pd.read_csv('gender_submission.csv')


all_data = pd.concat([train, test], ignore_index=True)
all_data = all_data.dropna(subset='Embarked')
all_data['Age'] = all_data['Age'].fillna(-1)
all_data['Fare'] = all_data['Fare'].fillna(all_data['Fare'].median())

# # split cabin to number and letter(floor)
all_data['Floor'] = all_data['Cabin'].str.extract('([a-zA-Z]+)', expand=False)
all_data['Room_Num'] = all_data['Cabin'].str.extract('(\d+)', expand=False)
all_data['Room_Num'] = all_data['Room_Num'].fillna(-1).astype(int)
all_data['Floor'] = all_data['Floor'].fillna(-1)
all_data = all_data.drop(columns=['Cabin', 'Name','PassengerId'])

all_data.isna().sum()


# label encoding
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

le = LabelEncoder()
all_data['Sex'] = le.fit_transform(all_data['Sex'])
all_data['Ticket'] = le.fit_transform(all_data['Ticket'])
# FLOOR NUMBER
category_mapping = {'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5,'F': 6,'G': 7,'T': 8, -1:-1}

# Use the map function to replace categories with numerical values
all_data['Floor'] = all_data['Floor'].map(category_mapping)


# One Hot Encoding
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), ['Embarked'])], remainder='passthrough')
all_data = pd.DataFrame(ct.fit_transform(all_data))


# split the data - remove passenger id
# x_train = all_data[0:len(train)-2,:]
# y_train = [0:len(train)-2,0]

Survived_col = 3
nan_indexes = np.where(np.isnan(all_data.iloc[:, Survived_col]))[0][0]
limit = nan_indexes
x_train = all_data.iloc[0:limit,:]
y_train = all_data.iloc[0:limit,Survived_col]

x_test = all_data.iloc[limit:,:]
y_test = all_data.iloc[limit:,Survived_col]

# remove Survived_col from x
x_test = x_test.drop(columns=Survived_col)
x_train = x_train.drop(columns=Survived_col)



In [ ]:
# ------------------------------ANN MODEL------------------------------------
from sklearn.model_selection import train_test_split

# scaling data - normilizing
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.fit_transform(x_test)

#Training

#Init the ANN
ann = tf.keras.models.Sequential()

ann.add(tf.keras.layers.Dense(units=6, activation='relu'))
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

ann.compile(optimizer='adam', loss='binary_crossentropy', metrics= ['accuracy'])

# Actual training
ann.fit(x_train, y_train, batch_size=32, epochs=100)

y_pred = ann.predict(x_test)

from sklearn.metrics import multilabel_confusion_matrix,accuracy_score, confusion_matrix

y_binary_pred = np.where(y_pred > 0.5, 1, 0)

y_test = y_test_data['Survived']
cm = confusion_matrix(y_test,y_binary_pred)
acc = accuracy_score(y_test,y_binary_pred)

print(cm)
print('acc is:', acc)

import csv

# Sample data
passenger_ids = test['PassengerId'].values

# Create a DataFrame from the given data
data = pd.DataFrame({
    'PassengerId': passenger_ids,
    'Survived': y_binary_pred.flatten()  # Flatten the array to match the shape
})

# Specify the file path
csv_file_path = 'output.csv'

# Write the DataFrame to a CSV file
data.to_csv(csv_file_path, index=False)

from google.colab import files

files.download('output.csv')




In [177]:
# ----------- new features
train = pd.read_csv('train (1).csv')
test = pd.read_csv('test (1).csv')
y_test_data = pd.read_csv('gender_submission.csv')

In [ ]:
# fill nan in age and cabin with -1
# drop passengerid, name,
# label encoder : sex, Ticket
# one hot encoded : embarked,
# split cabin to number and letter(floor) => label encoding to the letter

# tickets => split it to str and num
# @@ cabins => convert the floor to number c=3000...
# family members => SibSp	+ Parch + 1

array([-1, 'C', 'E', 'G', 'D', 'A', 'B', 'F', 'T'], dtype=object)

In [206]:
all_data = pd.concat([train, test], ignore_index=True)
all_data = all_data.dropna(subset='Embarked')
all_data['Age'] = all_data['Age'].fillna(-1)
all_data['Fare'] = all_data['Fare'].fillna(all_data['Fare'].median())

# # split cabin to number and letter(floor)
all_data['Floor'] = all_data['Cabin'].str.extract('([a-zA-Z]+)', expand=False)
all_data['Room'] = all_data['Cabin'].str.extract('(\d+)', expand=False)
all_data['Room'] = all_data['Room'].fillna(-1).astype(int)
all_data['Floor'] = all_data['Floor'].fillna(-1)

# Mapping dictionary for the alphabetic part
floor_mapping = {'A': 1000, 'B': 2000, 'C': 3000, 'D': 4000, 'E': 5000, 'F': 6000, 'G': 7000, 'H': 8000}

# Map alphabetic part to numerical values
all_data['Room_Num'] = all_data['Floor'].map(floor_mapping) + all_data['Room']
all_data['Room_Num'] = all_data['Room_Num'].fillna(-1)


all_data['Tick_str'] = all_data['Ticket'].str.extract('([a-zA-Z]+)', expand=False)
all_data['Tick_num'] = all_data['Ticket'].str.extract('(\d+)', expand=False)
all_data['Tick_str'] = all_data['Tick_str'].fillna('')
all_data['Tick_num'] = all_data['Tick_num'].fillna(-1)

all_data['Fam_mem'] = all_data['SibSp'] + all_data['Parch'] + 1

all_data = all_data.drop(columns=['Cabin','Name','PassengerId','Ticket'])

all_data.isna().sum()


Survived    418
Pclass        0
Sex           0
Age           0
SibSp         0
Parch         0
Fare          0
Embarked      0
Floor         0
Room          0
Room_Num      0
Tick_str      0
Tick_num      0
Fam_mem       0
dtype: int64

In [207]:
# label encoding
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

le = LabelEncoder()
all_data['Sex'] = le.fit_transform(all_data['Sex'])
all_data['Tick_str'] = le.fit_transform(all_data['Tick_str'])
# FLOOR NUMBER
category_mapping = {'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5,'F': 6,'G': 7,'T': 8, -1:-1}

# Use the map function to replace categories with numerical values
all_data['Floor'] = all_data['Floor'].map(category_mapping)


In [208]:
# One Hot Encoding
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), ['Embarked'])], remainder='passthrough')
all_data = pd.DataFrame(ct.fit_transform(all_data))

In [210]:
# split the data - remove passenger id
# x_train = all_data[0:len(train)-2,:]
# y_train = [0:len(train)-2,0]

Survived_col = 3
nan_indexes = np.where(pd.isna(all_data.iloc[:,Survived_col]))[0][0]

limit = nan_indexes
x_train = all_data.iloc[0:limit,:]
y_train = all_data.iloc[0:limit,Survived_col]

x_test = all_data.iloc[limit:,:]
y_test = all_data.iloc[limit:,Survived_col]

# remove Survived_col from x
x_test = x_test.drop(columns=Survived_col)
x_train = x_train.drop(columns=Survived_col)

print(np.where(pd.isna(all_data.iloc[:,Survived_col]))[0][-1])

1306


In [212]:
# ------------------------------ANN MODEL------------------------------------
from sklearn.model_selection import train_test_split


# scaling data - normilizing
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.fit_transform(x_test)

x_train_tensor = tf.convert_to_tensor(x_train, dtype=tf.float64)
y_train_tensor = tf.convert_to_tensor(y_train, dtype=tf.float64)

#Training

#Init the ANN
ann = tf.keras.models.Sequential()

ann.add(tf.keras.layers.Dense(units=6, activation='relu'))
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

ann.compile(optimizer='adam', loss='binary_crossentropy', metrics= ['accuracy'])

# Actual training
ann.fit(x_train_tensor, y_train_tensor, batch_size=32, epochs=100)

y_pred = ann.predict(x_test)


Epoch 1/100
28/28 [==============================] - 2s 39ms/step - loss: 0.6668 - accuracy: 0.6513
Epoch 2/100
28/28 [==============================] - 0s 2ms/step - loss: 0.6341 - accuracy: 0.6625
Epoch 3/100
28/28 [==============================] - 0s 2ms/step - loss: 0.6043 - accuracy: 0.6828
Epoch 4/100
28/28 [==============================] - 0s 4ms/step - loss: 0.5766 - accuracy: 0.7042
Epoch 5/100
28/28 [==============================] - 0s 4ms/step - loss: 0.5536 - accuracy: 0.7098
Epoch 6/100
28/28 [==============================] - 0s 5ms/step - loss: 0.5351 - accuracy: 0.7143
Epoch 7/100
28/28 [==============================] - 0s 3ms/step - loss: 0.5203 - accuracy: 0.7244
Epoch 8/100
28/28 [==============================] - 3s 100ms/step - loss: 0.5095 - accuracy: 0.7334
Epoch 9/100
28/28 [==============================] - 0s 3ms/step - loss: 0.5010 - accuracy: 0.7503
Epoch 10/100
28/28 [==============================] - 0s 3ms/step - loss: 0.4938 - accuracy: 0.7503
Epoch 

In [213]:
from sklearn.metrics import multilabel_confusion_matrix,accuracy_score, confusion_matrix

y_binary_pred = np.where(y_pred > 0.5, 1, 0)

y_test = y_test_data['Survived']
cm = confusion_matrix(y_test,y_binary_pred)
acc = accuracy_score(y_test,y_binary_pred)

In [214]:
print(cm)
print('acc is:', acc)


[[239  27]
 [ 30 122]]
acc is: 0.8636363636363636


In [215]:
import csv

# Sample data
passenger_ids = test['PassengerId'].values

# Create a DataFrame from the given data
data = pd.DataFrame({
    'PassengerId': passenger_ids,
    'Survived': y_binary_pred.flatten()  # Flatten the array to match the shape
})

# Specify the file path
csv_file_path = 'output.csv'

# Write the DataFrame to a CSV file
data.to_csv(csv_file_path, index=False)

from google.colab import files

files.download('output.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [44]:
print(y_binary_pred.sum())
print(y_test_data['Survived'].sum())
true_inf = (y_binary_pred.flatten()==y_test_data['Survived'].values).sum()
print(true_inf/418)

136
152
0.8516746411483254


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>